In [3]:
from datetime import datetime
import os, subprocess, json, requests, shutil, time

In [4]:
def check_queue(queue_name):
    rabbitmq_host = os.environ.get('rabbitmq_host','http://192.168.1.110')
    rabbitmq_port = os.environ.get('rabbitmq_port','32311')
    user = os.environ.get('user','celery')
    password = os.environ.get('password','celery')
    url = rabbitmq_host + ':' + str(rabbitmq_port) + '/api/queues/celery/' + queue_name
    print ('Checking RabbitMQ queue depth for: ' + queue_name)
    worker_queue = json.loads(requests.get(url, auth=(user,password)).text)
    queue_depth = (worker_queue["messages_unacknowledged"])
    return queue_depth

In [5]:
def find_files(directory, extensions):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.endswith(ext) for ext in extensions):
                file_path = os.path.join(root, file)
                yield root, file, file_path



In [19]:
directory = 'C:\\Users\\Chase\\Documents\\GitHub\\Boilest\\Scripts\\Configurations'
extensions = '.json'
queue_depth = check_queue('worker')
print ('Current Worker queue depth is: ' + str(queue_depth))

if queue_depth == 0:
       for root, file, file_path in find_files(directory,extensions):
           print (file_path)
           f = open(file_path)
           json_configuration = json.load(f)
           if json_configuration["enabled"] == 'true':
               print (json.dumps(json_configuration, indent=3, sort_keys=True))
               print ('sending ' + json_configuration["config_name"] + ' to ffinder')
           elif json_configuration["enabled"] == 'false':
               print ('Not condsidering ' + json_configuration["config_name"] + ' at this time') 
           else:
                print ('Did not find Configurations')

Checking RabbitMQ queue depth for: worker
Current Worker queue depth is: 0
C:\Users\Chase\Documents\GitHub\Boilest\Scripts\Configurations\audio\Anime_OPUS_Audio.json
Not condsidering Anime_OPUS_Starter at this time
C:\Users\Chase\Documents\GitHub\Boilest\Scripts\Configurations\audio\Movies_OPUS_Audio.json
Not condsidering Movies_OPUS_Starter at this time
C:\Users\Chase\Documents\GitHub\Boilest\Scripts\Configurations\audio\TV_OPUS_Audio .json
Not condsidering TV_OPUS_Starter at this time
C:\Users\Chase\Documents\GitHub\Boilest\Scripts\Configurations\container\Anime_MKV_Container.json
{
   "config_name": "Anime_MKV_Container_Check",
   "enabled": "true",
   "ffmpeg_codec_type": "container",
   "ffmpeg_string": "-c copy",
   "ffprobe_string": "ffprobe -loglevel quiet -show_entries format -of json",
   "format_extension": ".mkv",
   "format_name": "matroska,webm",
   "notes": "Convering all video containers to MKV",
   "override": "true",
   "watch_folder": "/anime"
}
sending Anime_MKV_Con

In [20]:
def ffprobe(ffprobe_string,file_path):
    subprocess_cmd = ffprobe_string + ' "' + file_path + '"'
    p = subprocess.run(subprocess_cmd, capture_output=True, text=True).stdout
    d = json.loads(p)
    return d    

In [27]:
import subprocess
import json

ffprobe_command = 'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout -of json'
file_path = 'C:\\Users\\Chase\\media\\Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4'

def run_ffprobe(ffprobe_command):
    try:
        result = subprocess.run(ffprobe_command, capture_output=True, text=True, shell=True, check=True)
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        return {"error": f"Error running ffprobe: {e}"}

# Example usage:
video_file_path = 'C:\\Users\\Chase\\media\\Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4'

# Define the ffprobe command as a single string
ffprobe_command = f'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout -of json "{video_file_path}"'

# Run the ffprobe command and get the result in JSON format
result_json = run_ffprobe(ffprobe_command)

print(json.dumps(result_json, indent=2))


{
  "programs": [],
  "streams": [
    {
      "index": 0,
      "codec_name": "h264",
      "codec_type": "video"
    },
    {
      "index": 1,
      "codec_name": "aac",
      "codec_type": "audio",
      "channel_layout": "stereo"
    }
  ],
  "format": {
    "filename": "C:\\Users\\Chase\\media\\Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4",
    "nb_streams": 2,
    "nb_programs": 0,
    "format_name": "mov,mp4,m4a,3gp,3g2,mj2",
    "format_long_name": "QuickTime / MOV",
    "start_time": "0.000000",
    "duration": "680.279600",
    "size": "17701382",
    "bit_rate": "208165",
    "probe_score": 100,
    "tags": {
      "major_brand": "mp42",
      "minor_version": "512",
      "compatible_brands": "isomiso2avc1mp41",
      "creation_time": "2018-04-09T08:42:52.000000Z",
      "title": "Aqua Teen Hunger Force S01E03 Bus of the Undead",
      "encoder": "HandBrake 0.10.5 2016021100"
    }
  }
}


In [31]:
import subprocess
import json

ffprobe_command = 'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout -of json'
file_path = 'C:\\Users\\Chase\\media\\Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4'

def ffprober2(ffprobe_command,file_path):
    try:
        full_command = f'{ffprobe_command} "{file_path}"'
        result = subprocess.run(full_command, capture_output=True, text=True, shell=True, check=True)
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        return {"error": f"Error running ffprobe: {e}"}

result_json = ffprober2(ffprobe_command, file_path)
print(json.dumps(result_json, indent=2))

{
  "programs": [],
  "streams": [
    {
      "index": 0,
      "codec_name": "h264",
      "codec_type": "video"
    },
    {
      "index": 1,
      "codec_name": "aac",
      "codec_type": "audio",
      "channel_layout": "stereo"
    }
  ],
  "format": {
    "filename": "C:\\Users\\Chase\\media\\Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4",
    "nb_streams": 2,
    "nb_programs": 0,
    "format_name": "mov,mp4,m4a,3gp,3g2,mj2",
    "format_long_name": "QuickTime / MOV",
    "start_time": "0.000000",
    "duration": "680.279600",
    "size": "17701382",
    "bit_rate": "208165",
    "probe_score": 100,
    "tags": {
      "major_brand": "mp42",
      "minor_version": "512",
      "compatible_brands": "isomiso2avc1mp41",
      "creation_time": "2018-04-09T08:42:52.000000Z",
      "title": "Aqua Teen Hunger Force S01E03 Bus of the Undead",
      "encoder": "HandBrake 0.10.5 2016021100"
    }
  }
}


In [320]:
configuration_json = {
  "config_name": "Anime_MKV_Starter",
  "watch_folder": "C:\\Users\\Chase\\media",
  "format_name": "matroska,webm",
  "ffmpeg_string": ".mkv",
  "ffmpeg_codec_type": "container",
  "ffprobe_string":"ffprobe -loglevel quiet -show_entries format -of json",
  "notes":"",
  "override":"true",
  "enabled":"true"
}



extensions = [".mp4", ".mkv", ".avi"]
for root, file, file_path in find_files(configuration_json["watch_folder"], extensions):
    if file.endswith(configuration_json["ffmpeg_string"]):
        print(file + ' has an ' + configuration_json["ffmpeg_string"] + ' file extension, no action to take.')
    else:
        print(file + ' does not have a ' + configuration_json["ffmpeg_string"] + ' file extension')
        output_filename = os.path.splitext(file)[0] + configuration_json["ffmpeg_string"]
        print ('output_filename is: ' + output_filename)
        print ('file is: ' + file)
        print ('root is: ' + root)
        print ('file_path is: ' + file_path)

        
        
        
        
        
     

Aqua Teen Hunger Force - S01E02 - Escape from Leprechaupolis.mp4 does not have a .mkv file extension
output_filename is: Aqua Teen Hunger Force - S01E02 - Escape from Leprechaupolis.mkv
file is: Aqua Teen Hunger Force - S01E02 - Escape from Leprechaupolis.mp4
root is: C:\Users\Chase\media
file_path is: C:\Users\Chase\media\Aqua Teen Hunger Force - S01E02 - Escape from Leprechaupolis.mp4
Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4 does not have a .mkv file extension
output_filename is: Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mkv
file is: Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4
root is: C:\Users\Chase\media
file_path is: C:\Users\Chase\media\Aqua Teen Hunger Force - S01E03 - Bus of the Undead.mp4
Aqua Teen Hunger Force - S01E04 - Mayhem of the Mooninites.mp4 does not have a .mkv file extension
output_filename is: Aqua Teen Hunger Force - S01E04 - Mayhem of the Mooninites.mkv
file is: Aqua Teen Hunger Force - S01E04 - Mayhem of the Mooninites.mp4
ro

In [274]:
def file_size_mb(file_path):
    # Used a bit in tasks_worker
    file_size = round(os.stat(file_path).st_size / (1024 * 1024))
    return file_size

In [292]:
ffmpeg_settings = os.environ.get('ffmpeg_settings','-hide_banner -loglevel 16 -stats -stats_period 10')
file_path = 'C:\\Users\\Chase\\media\\test test.mp4'
root = 'C:\\Users\\Chase\\media'
file = 'test test.mp4'
output_file = 'test test.mkv'
temp_filepath = 'C:\\Users\\Chase\\media\\temp\\' +  output_file
destination_filepath = root + '\\' + output_file
ffmpeg_command = '-c copy'


ffmpeg_command = 'ffmpeg ' + ffmpeg_settings + ' -i "' + file_path + '" ' + ffmpeg_command + ' "' + temp_filepath + '"'
print (ffmpeg_command)

print ('Please hold')
process = subprocess.Popen(ffmpeg_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True)
for line in process.stdout:
    print(line)

if os.path.exists(file_path and temp_filepath):
    file_path_stats = file_size_mb(file_path)
    print ('old ' + file + ' was: '+ str(file_path_stats))
    temp_filepath_stats = file_size_mb(file_path)
    print ('new ' + output_file + ' is: '+ str(temp_filepath_stats))
    encoded_stats_difference = file_path_stats - temp_filepath_stats
    print ('Spave saved was: '+ str(encoded_stats_difference))
    
    if temp_filepath_stats != 0.0:
        destination_filepath = root + '\\' + output_file
        print('Moving ' + temp_filepath + ' to ' + destination_filepath)
        os.remove(file_path)
        shutil.move(temp_filepath, destination_filepath) 

else:
    print ('Something went wrong, a file is missing')
    


    
    



ffmpeg -hide_banner -loglevel 16 -stats -stats_period 10 -i "C:\Users\Chase\media\test test.mp4" -c copy "C:\Users\Chase\media\temp\test test.mkv"
Please hold
frame=    0 fps=0.0 q=-1.0 size=       0kB time=-00:00:00.06 bitrate=  -0.0kbits/s speed=N/A    

frame=20796 fps=0.0 q=-1.0 Lsize=   21090kB time=00:11:33.84 bitrate= 249.0kbits/s speed=2.03e+03x    

old test test.mp4 was: 21
new test test.mkv is: 21
Spave saved was: 0
Moving C:\Users\Chase\media\temp\test test.mkv to C:\Users\Chase\media\test test.mkv


In [308]:
def check_queue(queue_name):
    rabbitmq_host = os.environ.get('rabbitmq_host','http://192.168.1.110')
    rabbitmq_port = os.environ.get('rabbitmq_port','32311')
    user = os.environ.get('user','celery')
    password = os.environ.get('password','celery')
    url = rabbitmq_host + ':' + str(rabbitmq_port) + '/api/queues/celery/' + queue_name
    print ('Checking RabbitMQ queue depth for: ' + queue_name)
    worker_queue = json.loads(requests.get(url, auth=(user,password)).text)
    queue_depth = (worker_queue["messages_unacknowledged"])
    return queue_depth
depth=check_queue('worker')
print(depth)

Checking RabbitMQ queue depth for: worker
0


In [307]:
my_dict = {'key1': 'value1', 'key2': 'value2'}

# Removing an item
del my_dict['key1']

print (my_dict)

{'key2': 'value2'}


In [1]:

extensions = [".mp4", ".mkv", ".avi"]
extensions.remove('.mkv')
print (extensions)




['.mp4', '.avi']


In [3]:
json_configuration = ({'ffmpeg_codec_type':'container','format_extension':'.mkv'})


if json_configuration["ffmpeg_codec_type"] == 'container':
    extensions = [".mp4", ".mkv", ".avi"]
    extensions.remove(json_configuration['format_extension'])
else:
    extensions = json_configuration["format_extension"]

print (extensions)

['.mp4', '.avi']


In [6]:
import os
json_configuration = ({'file':'file.avi','format_extension':'.mkv'})

output_filename = os.path.splitext(json_configuration["file"])[0] + json_configuration["format_extension"]

print('Setting up: ' + json_configuration["file"] + ' as ' + output_filename)

Setting up: file.avi as file.mkv


In [1]:
file_located = {'file':'file.mp4'}
if file_located['file'].split('.')[-1] != '.mkv':
    print ('hi')

hi


In [2]:
import os

def get_file_name_from_path(file_path):
    return os.path.basename(file_path)

# Example usage:
file_path = "/path/to/your/directory/your_file.txt"
file_name = get_file_name_from_path(file_path)
print("File Name:", file_name)

File Name: your_file.txt


In [1]:
import json

my_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
my_string = "This is a string with the dictionary: " + json.dumps(my_dict)

print(my_string)

This is a string with the dictionary: {"key1": "value1", "key2": "value2", "key3": "value3"}


In [5]:
import json

def get_json():
    result_dict = {
        'directory': 'directory',
        'root': 'root222',
        'file': 'file',
        'file_path': 'file_path',
        'extension': 'extension'
    }
    return json.dumps(result_dict)

stuff = get_json()
stuff = json.loads(stuff)
print(json.dumps(stuff, indent=3, sort_keys=True))

print (stuff['root'])

{
   "directory": "directory",
   "extension": "extension",
   "file": "file",
   "file_path": "file_path",
   "root": "root222"
}
root222
